# Anime Analysis 

#### Source: https://www.anime-planet.com/ 
##### The best place to find new anime to enjoy!

#### Before we can start, we need to import the required packages: 

In [2]:
import sys
import re
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool
import numpy as np
import pandas as pd

In [106]:
#exist = pd.read_pickle("anime_list.csv")
missing_values = ["na", "['na']", "['nan']"]
exist = pd.read_csv('./anime_list.csv', na_values=missing_values, sep=';')

In [107]:
sys.setrecursionlimit(0x100000)

#### Then we can scrape the data from the website: 

In [108]:
r = requests.get('https://www.anime-planet.com/anime/all?page=1&bvm=list')
soup = BeautifulSoup(r.text, 'lxml')

#### Let's take a look at what we got: 

In [109]:
soup

<!DOCTYPE html>
<html class="no-js" lang="en" xml:lang="en">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# profile: http://ogp.me/ns/profile# book: http://ogp.me/ns/book# video: http://ogp.me/ns/video#">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Browse All Anime | Anime-Planet</title>
<meta content="Create of a list of anime you've seen, watch them online, discover new anime and more on Anime-Planet. Search thousands of anime by your favorite tags and genres, studios, years, ratings, and more!" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png?v=WGowMEAKpM" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png?v=WGowMEA

#### Crap... what have I gotten myself into?..   
#### Oh wait.. Is it a bird? Is it a plane? No, it's Beautiful Soup! To the rescue!

In [110]:
title, media_format, url = ([] for i in range(3))

In [111]:
def scraper(i):
    
    print(i)
    #global title, media_format, url
    
    page = 'https://www.anime-planet.com/anime/all?page=' + str(i) + '&bvm=list'
    r = requests.get(page)
    soup = BeautifulSoup(r.text, 'lxml')
    
    #if soup.find_all('td', attrs={'class': "tableTitle"}) not in exist['Title']:
    
    if soup.find_all('td', attrs={'class': "tableTitle"}):
        #title += soup.find_all('td', attrs={'class': "tableTitle"})
        title = soup.find_all('td', attrs={'class': "tableTitle"})
    else:
        title = 'na'



    if soup.find_all('td', attrs={'class': "tableType"}):
        #media_format += soup.find_all('td', attrs={'class': "tableType"})
        media_format = soup.find_all('td', attrs={'class': "tableType"})
    else:
        #media_format.append('na')
        media_format = 'na'

    url = []
    for section in soup.find_all('td', attrs={'class': "tableTitle"}): 
        extract = BeautifulSoup(str(section))
        if extract.find('a', {'data-tooltip-entry':'anime'})['href']:
            mini = 'https://www.anime-planet.com' + extract.find('a', {'data-tooltip-entry':'anime'})['href']
            url.append(mini)
        else:
            url.append('na')

    return [title, media_format, url]
    
    


In [112]:
p = Pool()
title, media_format, url = zip(*p.map(scraper, range(1,372)))
p.terminate()
p.join()

1
37
25
13
49
61
73
85
26
2
74
62
38
50
86
14
27
39
75
63
51
87
15
3
28
40
16
64
4
76
52
88
29
41
17
77
5
53
65
89
42
30
18
6
66
78
54
90
31
43
19
7
67
79
91
55
32
44
20
8
68
80
56
92
33
45
9
21
57
81
69
93
34
46
10
22
58
82
94
70
35
11
47
23
59
36
71
83
95
12
48
24
60
72
96
84
97
109
121
98
103
110
122
99
111
123
100
112
124
101
113
125
102
114
126
133
115
127
104
145
157
134
169
116
128
146
105
181
158
135
170
117
129
147
106
182
159
136
118
171
130
148
107
183
160
137
131
108
149
119
172
184
161
138
132
150
120
162
185
173
139
151
163
186
140
174
152
164
141
175
187
153
165
142
188
176
154
166
143
177
189
155
167
144
178
190
193
156
168
179
194
191
180
195
192
196
197
198
199
205
200
217
206
201
218
207
229
202
219
208
203
230
241
220
253
209
231
204
265
242
221
277
254
210
232
266
243
289
255
278
222
211
233
267
244
290
256
279
223
212
234
268
245
257
224
291
280
213
235
258
292
214
281
236
269
246
225
259
215
293
237
226
270
247
282
260
216
238
294
248
271
283
227
261
239
295
272


In [113]:
title[0][0].text

'Fullmetal Alchemist: Brotherhood'

In [114]:
tmp_title = title
tmp_format = media_format
tmp_url = url
title, media_format, url = ([] for i in range(3))
for i in range(0, len(tmp_title)):
    for j in range(0, len(tmp_title[i])):
        if (tmp_title[i][j]).text not in exist['Title'].values:
            title.append(tmp_title[i][j])
            media_format.append(tmp_format[i][j])
            url.append(tmp_url[i][j])

In [115]:
for i in range(len(title)):
    title[i] = title[i].text
    media_format[i] = media_format[i].text

In [116]:
title[0:10]

['Happy George']

In [117]:
len(url)

1

In [118]:
alt, eps, studio, year_range, desc, tags, acc_rating, votes = ([] for i in range(8))

In [119]:
count = 1
def get_each(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    global count
    print(count)
    count += 1

    if soup.find_all('h2', attrs={'class': "aka"}):
        #alt += soup.find_all('h2', attrs={'class': "aka"})
        alt = (soup.find_all('h2', attrs={'class': "aka"})[0].text)[12:-1]
    else:
        #alt.append('na')
        alt = 'na'
        
    if soup.find_all('span', attrs={'class': "type"}):
        #eps += soup.find_all('span', attrs={'class': "type"})
        eps = soup.find_all('span', attrs={'class': "type"})[0].text
    else:
        #eps.append('na')
        eps = 'na'
        
    studio = []
    if soup.find_all('a', {"href" : lambda L: L and L.startswith('/anime/studios/')}):
        #stds = []
        #stds = soup.find_all('a', {"href" : lambda L: L and L.startswith('/anime/studios/')})
        stds = soup.find_all('a', attrs={"href" : lambda L: L and L.startswith('/anime/studios/')})
        
        for i in range(0, len(stds)):
            studio.append(stds[i].text)
            
        '''if len(stds) > 1:
            #studio.append(stds)
            studio = stds
            print(studio)
        else:
            #studio += stds
            studio = stds
            print(studio)'''
    else:
        studio = 'na'
        
    if soup.find_all('span', attrs={'class': "iconYear"}):
        #year_range += soup.find_all('span', attrs={'class': "iconYear"})
        year_range = soup.find_all('span', attrs={'class': "iconYear"})[0].text
    else:
        #year_range.append('na')
        year_range = 'na'
        
    
    if soup.find_all('div', attrs={'itemprop': 'description'}):
        #desc += soup.find_all('div', attrs={'itemprop': 'description'})
        desc = soup.find_all('div', attrs={'itemprop': 'description'})[0].text
    else:
        #desc.append('na')
        desc = 'na'
    
    if soup.find_all('a', attrs={'data-tooltip': 'tags'}):
        #tags.append(soup.find_all('a', attrs={'data-tooltip': 'tags'}))
        tags = soup.find_all('a', attrs={'data-tooltip': 'tags'})
        
        for i in range(0, len(tags)):
            tags[i] = (tags[i].text)[1:-1]
        
    else:
        #tags.append('na')
        tags = 'na'
        
    acc_rating, votes = ([] for i in range(2))
    
    for section in soup.find_all('div', attrs={'class': "avgRating"}):
        extract = BeautifulSoup(str(section))
        
        if extract.find('meta', {'itemprop':'ratingValue'}):
            #acc_rating.append(extract.find('meta', {'itemprop':'ratingValue'})['content'])
            acc_rating = float(extract.find('meta', {'itemprop':'ratingValue'})['content'])
        else:
            acc_rating.append('na')                            

        if extract.find('meta', {'itemprop':'ratingCount'}):
            #votes.append(extract.find('meta', {'itemprop':'ratingCount'})['content'])
            votes = int(extract.find('meta', {'itemprop':'ratingCount'})['content'])
        else:
            votes.append('na')
            
    return [alt, eps, studio, year_range, desc, tags, acc_rating, votes ]

In [120]:
get_each('https://www.anime-planet.com/anime/one-piece-special-protect-the-last-great-stage')

1


['One Piece Special 3: Mamore! Saigo no Dai Butai',
 'TV Special (1 ep x 46 min)',
 ['Toei Animation'],
 ' 2003 ',
 'The renowned Randolph troupe is in town and they are putting on a special farewell play to mark their leader’s retirement. When three of its actors quit a mere thirty minutes before the curtain rises, the Straw Pirates offer their services; though with Robyn’s overacting, Usopp trying to hog the limelight and Luffy gate-crashing the stage dressed as a monkey, the first act becomes more of a farce instead! Before the second act can commence, Randolph is arrested by the marines for selling stolen weapons to pirates, and boards the ship to prove his innocence. There, he’s confronted by the Marine Commander named Governor, a former subordinate of Randolph’s, who’s concocted the whole plan in order to exact his revenge on the ex-marine. Now, in addition to rescuing Randolph and protecting his ship, Luffy must also convince the troupe leader that not all pirates are bloodthirs

In [121]:
p = Pool()
alt, eps, studio, year_range, desc, tags, acc_rating, votes  = zip(*p.map(get_each, url))
p.terminate()
p.join()

2


In [122]:
alt = list(alt)
eps = list(eps)
studio = list(studio)
year_range = list(year_range)
desc = list(desc)
tags = list(tags)
acc_rating = list(acc_rating)
votes = list(votes)

In [123]:
year_range[-10:-1]

[]

In [124]:
print(acc_rating[0])
print(acc_rating[-1])

['na']
['na']


In [125]:
len(acc_rating)

1

In [126]:
dur, comp = ([] for i in range(2))

for i in range(len(eps)):
    nums = re.findall("(\d+)", str(eps[i]))
    
    if eps[i].find('+') != -1 or year_range[i].find('TBA') != -1 or year_range[i].find('?') != -1:
        comp.append(0)
    else:
        comp.append(1)
        
    if len(nums) == 1:
        eps[i] = int(nums[0])
        dur.append('na')
    elif len(nums) > 1:
        eps[i] = int(nums[0])
        dur.append(int(nums[1]))
    else:
        eps[i] = 'na'
        dur.append('na')
            
            

In [127]:
df = pd.DataFrame({'Rating': acc_rating, 'Votes': votes, 'Title':title, 'Alt Title':alt, 'Format':media_format, 'Episodes':eps, 'Duration':dur, 'Completed':comp, 'Studio':studio, 'Year':year_range, 'Tags':tags, 'Description':desc})

In [128]:
df

,Rating,Votes,Title,Alt Title,Format,Episodes,Duration,Completed,Studio,Year,Tags,Description
0,[na],[na],Happy George,na,TV,52,1,1,[Kachidoki Studio],2015 - 2016,"[Comedy, Family Life, Short Episodes, Flash An...",na


In [129]:
df.loc[0,'Tags'][0]

'Comedy'

In [65]:
####df.to_csv("anime_list.csv", sep=';', index=False)

In [130]:
df.to_csv("anime_list.csv", sep=';', index=False, mode='a', header=False)

In [3]:
missing_values = ["na", "['na']", "['nan']"]
df = pd.read_csv('./anime_list.csv', na_values=missing_values, sep=';', header=0)

In [4]:
df['Rating'] = pd.to_numeric(df['Rating'],errors='coerce')
df['Votes'] = pd.to_numeric(df['Votes'],errors='coerce')
df['Episodes'] = pd.to_numeric(df['Episodes'],errors='coerce')
df['Duration'] = pd.to_numeric(df['Duration'],errors='coerce')

In [5]:
def start_end_years(years):
    start_years = []
    end_years = []
    for i in range(0, len(years)):
        if years[i].find('TBA') != -1:
            start_years.append('TBA')
            end_years.append('TBA')
        elif years[i].find('-') != -1:
            if years[i].find('?') != -1:
                start_years.append(re.findall('\d+', years[i])[0])
                end_years.append('?')
            else:
                start_years.append(re.findall('\d+', years[i])[0])
                end_years.append(re.findall('\d+', years[i])[1])
        else:
            if re.findall('\d+', years[i]):
                start_years.append(re.findall('\d+', years[i])[0])
                end_years.append(re.findall('\d+', years[i])[0])
            
    return start_years, end_years

start_years, end_years = start_end_years(list(df['Year']))

start_years = pd.Series(start_years, name='Start Year')
end_years = pd.Series(end_years, name='End Year')

df = pd.concat([df, start_years], axis=1)
df = pd.concat([df, end_years], axis=1)

df.drop(['Year'], axis=1, inplace=True)

cols = ['Rating','Votes','Title','Alt Title','Format','Episodes','Duration' ,'Start Year','End Year','Completed','Studio','Tags','Description']
df = df[cols]

In [6]:
def split_list(stringy):
    if stringy != 'nan':
        stringy = str(stringy).replace("'", "").replace("[", "").replace( "]", "")
        return str(stringy).split(', ')
    else:
        return np.NaN

df['Studio'] = df['Studio'].apply(split_list)
df['Tags'] = df['Tags'].apply(split_list)

In [7]:
def backslash_remover(each):
    listy = []
    if type(each) != float:
        for tag in list(each):
            listy.append(tag.replace("\\n", "").replace("\\", ""))
        return listy
    else:
        return np.NaN

df['Tags'] = df['Tags'].apply(backslash_remover)

In [8]:
def spaceRemover(tags):
    tags_list = []
    if type(tags) != float:
        for each in tags:
            if each[0] == ' ':
                tags_list.append(each[1:])
            else:
                tags_list.append(each)
        return tags_list
    else:
        return np.NaN
    
df['Tags'] = df['Tags'].apply(spaceRemover)

In [9]:
def unwantedRemover(tags):
    tags_list = []
    if type(tags) != float:
        for each in tags:
            if each[0] == '"':
                if each[1] == ' ':
                    tags_list.append(each[2:-1])
                else:
                    tags_list.append(each[1:-1])
            else:
                tags_list.append(each)
        return tags_list
    else:
        return np.NaN
    
df['Tags'] = df['Tags'].apply(unwantedRemover)
df['Studio'] = df['Studio'].apply(unwantedRemover)

In [10]:
df.drop(['Completed'], axis=1, inplace=True)

In [11]:
def incompleteEndRemover(each):
    if type(each) != float:
        if each == 'TBA' or each == '?':
            return np.NaN
        else:
            return each
    else:
        return np.NaN

df['Start Year'] = df['Start Year'].apply(incompleteEndRemover)
df['End Year'] = df['End Year'].apply(incompleteEndRemover)

In [12]:
df.loc[df['Format'] == 'Movie', 'End Year'] = np.NaN

In [13]:
def nullifier(each):
    if 'nan' in each:
        return np.NaN    
    else:
        return each
    
df['Studio'] = df['Studio'].apply(split_list).apply(nullifier)
df['Tags'] = df['Tags'].apply(split_list).apply(nullifier)

df['Format'] = df['Format'].astype('category')
df['Start Year'] = df['Start Year'].astype('category')
df['End Year'] = df['End Year'].astype('category')

In [14]:
df

,Rating,Votes,Title,Alt Title,Format,Episodes,Duration,Start Year,End Year,Studio,Tags,Description
0,9.410,73986.0,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV,64.0,NaN,2009,2010,[Bones],"[Action, Adventure, Drama, Fantasy, Mystery, S...",The foundation of alchemy is based on the law ...
1,9.356,29247.0,your name.,Kimi no Na wa.,Movie,1.0,107.0,2016,NaN,[CoMix Wave Inc.],"[Drama, Romance, Body Swapping, Opposites Attr...",Mitsuha and Taki are two total strangers livin...
2,9.334,19592.0,A Silent Voice,Koe no Katachi,Movie,1.0,130.0,2016,NaN,[Kyoto Animation],"[Drama, Melancholy, School Life, Bullying, Mat...","After transferring into a new school, a deaf g..."
3,9.320,11195.0,Haikyuu!! Karasuno High School vs Shiratorizaw...,Haikyuu!! 3rd Season: Karasuno High vs. Shira...,TV,10.0,NaN,2016,2016,[Production I.G],"[Shounen, Sports, Animeism, School Club, Schoo...","Picking up where the second season ended, the ..."
4,9.296,18788.0,My Hero Academia 3,Boku no Hero Academia 3,TV,25.0,NaN,2018,2018,[Bones],"[Action, Comedy, Drama, Sci Fi, Shounen, Super...",Third season of My Hero Academia.
5,9.288,2939.0,Mob Psycho 100 II,NaN,TV,12.0,NaN,2019,NaN,[Bones],"[Action, Comedy, Shounen, Psychic Powers, Psyc...",Second Season of Mob Psycho 100.
6,9.274,15322.0,Haikyuu!! Second Season,NaN,TV,25.0,NaN,2015,2016,[Production I.G],"[Shounen, Sports, School Club, School Life, To...","After losing to Aoba Johsai at the Inter-High,..."
7,9.266,5375.0,Gintama Kanketsu-hen: Yorozuya yo Eien Nare,Gintama The Movie: The Final Chapter - Be Fore...,Movie,1.0,111.0,2013,NaN,[Sunrise],"[Action, Comedy, Drama, Sci Fi, Shounen, Alien...","While watching a movie, Gintoki comes upon a ""..."
8,9.264,29623.0,My Hero Academia 2,Boku no Hero Academia 2,TV,25.0,NaN,2017,2017,[Bones],"[Action, Comedy, Drama, Sci Fi, Shounen, Schoo...",In a world where eighty percent of the populat...
9,9.262,6192.0,Gintama (2015),Gintama°,TV,51.0,NaN,2015,2016,[BN Pictures],"[Action, Comedy, Drama, Sci Fi, Shounen, Alien...",Fourth season of Gintama.


In [15]:
def listStudios(std_col):
    uniq_std = []
    for std_row in std_col:
        if type(std_row) != float:
            for each in std_row:
                if each not in uniq_std:
                    uniq_std.append(each)
    return uniq_std

studios = listStudios(df['Studio'])

studios

['Bones',
 'CoMix Wave Inc.',
 'Kyoto Animation',
 'Production I.G',
 'Sunrise',
 'BN Pictures',
 'WHITE FOX',
 'MADHOUSE',
 'CloverWorks',
 'Shaft',
 'Artland',
 'Shuka',
 'Kinema Citrus',
 'Studio Chizu',
 'Studio Ghibli',
 'Studio VOLN',
 'David Production',
 'Brains Base',
 'Studio Deen',
 '8-Bit',
 'Tezuka Productions',
 'Mappa',
 'A-1 Pictures',
 'J.C. Staff',
 'ufotable',
 'Lerche',
 'G.CMay Animation & Film',
 'Studio Pierrot',
 'V1 Studio',
 'TMS Entertainment',
 'Magic Bus',
 'Bridge',
 'Wit Studio',
 'Synergy SP',
 'Toei Animation',
 'Graphinica',
 'Satelight',
 'P.A. Works',
 'Gainax',
 'Studio Khara',
 'Top Craft',
 'Studio Hibari',
 'Frederator Studios',
 'Tatsunoko Production',
 'XEBEC',
 'AIC',
 'Manglobe',
 'Double Eagle',
 'Studio Rikka',
 'Nippon Animation',
 'Ajia-do',
 'Geno Studio',
 'SILVER LINK',
 'Studio 4°C',
 'Hal Film Maker',
 'Actas',
 'feel.',
 'Studio NUT',
 'Lay-duce',
 'Telecom Animation Film',
 'A.C.G.T.',
 'IMAGIN',
 'GONZO',
 'Seven Arcs',
 'Orange',

In [16]:
df.to_pickle("anime_df.pkl")